In [1]:
import pandas as pd
import numpy as np
from get_vocabulary import get_vocabulary
from get_max_len import get_max_len

In [2]:
names_df = pd.read_csv('data/names.txt', sep="\n", header=None)
names_df.columns=['input']
names_df.head()

,input
0,John
1,William
2,James
3,Charles
4,George


### Preprocess names dataset

In [3]:
# Insert a tab in front of all the names
names_df['input'] = names_df['input'].apply(lambda x : '\t' + x)

# Append a newline at the end of every name
# We already appended a tab in front, so the target word should start at index 1
names_df['target'] = names_df['input'].apply(lambda x : x[1:len(x)] + '\n')
names_df.head()

,input,target
0,\tJohn,John\n
1,\tWilliam,William\n
2,\tJames,James\n
3,\tCharles,Charles\n
4,\tGeorge,George\n


- Now we have a DataFrame with two columns containing the names with the start and end tokens appended. The next step is to encode these as numeric values because machine learning models only accept numeric inputs.
- create two dictionaries, char_to_idx and idx_to_char, that will contain mappings of characters to integers, e.g., {'\t': 0, '\n': 1, 'a': 2, 'b': 3, ...} and the reverse mappings of integers to characters, e.g, {0: '\t', 1: '\n', 2: 'a', 3: 'b', ...}.

In [4]:
# Get the vocabulary
vocabulary = get_vocabulary(names_df['input'])

# Sort the vocabulary
vocabulary_sorted = sorted(vocabulary)

# Create the mapping of the vocabulary chars to integers
char_to_idx = { char : idx for idx, char in enumerate(vocabulary_sorted) }

# Create the mapping of the integers to vocabulary chars
idx_to_char = { idx : char for idx, char in enumerate(vocabulary_sorted) }

# Print the dictionaries
print(char_to_idx)
print(idx_to_char)

{'\t': 0, '\n': 1, 'A': 2, 'B': 3, 'C': 4, 'D': 5, 'E': 6, 'F': 7, 'G': 8, 'H': 9, 'I': 10, 'J': 11, 'K': 12, 'L': 13, 'M': 14, 'N': 15, 'O': 16, 'P': 17, 'Q': 18, 'R': 19, 'S': 20, 'T': 21, 'U': 22, 'V': 23, 'W': 24, 'X': 25, 'Y': 26, 'Z': 27, 'a': 28, 'b': 29, 'c': 30, 'd': 31, 'e': 32, 'f': 33, 'g': 34, 'h': 35, 'i': 36, 'j': 37, 'k': 38, 'l': 39, 'm': 40, 'n': 41, 'o': 42, 'p': 43, 'q': 44, 'r': 45, 's': 46, 't': 47, 'u': 48, 'v': 49, 'w': 50, 'x': 51, 'y': 52, 'z': 53}
{0: '\t', 1: '\n', 2: 'A', 3: 'B', 4: 'C', 5: 'D', 6: 'E', 7: 'F', 8: 'G', 9: 'H', 10: 'I', 11: 'J', 12: 'K', 13: 'L', 14: 'M', 15: 'N', 16: 'O', 17: 'P', 18: 'Q', 19: 'R', 20: 'S', 21: 'T', 22: 'U', 23: 'V', 24: 'W', 25: 'X', 26: 'Y', 27: 'Z', 28: 'a', 29: 'b', 30: 'c', 31: 'd', 32: 'e', 33: 'f', 34: 'g', 35: 'h', 36: 'i', 37: 'j', 38: 'k', 39: 'l', 40: 'm', 41: 'n', 42: 'o', 43: 'p', 44: 'q', 45: 'r', 46: 's', 47: 't', 48: 'u', 49: 'v', 50: 'w', 51: 'x', 52: 'y', 53: 'z'}


In [5]:
print(vocabulary)

{'T', 'R', 'C', 'n', 'p', 't', 'x', 'N', 'J', 'd', 'o', '\n', 'S', 'z', 'V', 'E', 'm', 'y', 'D', 'c', 'W', 'I', 'F', 'X', 'b', 'v', 'r', 'Q', 'L', 's', 'h', 'G', 'O', 'B', 'j', '\t', 'M', 'k', 'U', 'A', 'u', 'l', 'P', 'Z', 'g', 'q', 'i', 'f', 'K', 'Y', 'w', 'e', 'H', 'a'}


### Create input and target tensors
- The input is a list containing all the names in the dataset. So, the first dimension of the input tensor will be the number of names in the dataset. Each name can be thought of as a string having length equal to the length of the longest name and each character in each name is a one-hot encoded vector of size vocabulary. So, the second and third dimensions of the input tensor will be the length of the longest name and the size of the vocabulary. Similar is the case for the target tensor.

In [6]:
# Find the length of longest name
max_len = get_max_len(names_df['input'])

# Initialize the input vector
input_data = np.zeros((len(names_df['input']), max_len+1, len(vocabulary)), dtype='float32')

# Initialize the target vector
target_data = np.zeros((len(names_df['input']), max_len+1, len(vocabulary)), dtype='float32')

print(input_data.shape, target_data.shape, max_len)

(258000, 13, 54) (258000, 13, 54) 12


##### Now we have two vectors of appropriate shape which we can fill up with actual data. These vectors can then be fed to the recurrent neural network.

#### Initialize input and target vectors with values
- We created the input and target tensors of appropriate shape containing all zeros. Now, we'll fill these with actual values. The input and target tensors contain all the names in the dataset. Each name can be thought of as a string having length equal to the length of the longest name and each character in each name is a one-hot encoded vector of size vocabulary.
- The tensors can be filled-in as follows: `input_data[n_idx, p_idx, char_to_idx[char]]` will be set to 1 whenever the index of the name in the dataset is `n_idx` and it contains the character char in position `p_idx`

#### Example 3-d array

In [7]:
a = np.zeros((11,12,12))

In [8]:
a[1,1,10] = 1
a[1]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [9]:
# Iterate for each name in the dataset
for n_idx, name in enumerate(names_df['input']):
  # Iterate over each character and convert it to a one-hot encoded vector
  for c_idx, char in enumerate(name):
    input_data[n_idx, c_idx, char_to_idx[char]] = 1

# Iterate for each name in the dataset
for n_idx, name in enumerate(names_df['target']):
  # Iterate over each character and convert it to a one-hot encoded vector
  for c_idx, char in enumerate(name):
    target_data[n_idx, c_idx, char_to_idx[char]] = 1

##### Now we have the input and target vectors of appropriate shape. We can use these vectors to train the recurrent neural network.

### Build and compile RNN network
- We completed all the data preprocessing steps and have the input and target vectors ready. It is time to build the recurrent neural network. We'll create a small network architecture that will have 50 simple RNN nodes in the first layer followed by a dense layer. The dense layer will generate a probability distribution over the vocabulary for the next character. So, the size of the dense layer will be the same as the size of the vocabulary.

In [1]:
from keras.models import Sequential
from keras.layers import TimeDistributed, SimpleRNN, Dense, Activation

In [11]:
# Create a Sequential model
model = Sequential()

# Add SimpleRNN layer of 50 units
model.add(SimpleRNN(50, input_shape=(max_len+1, len(vocabulary)), return_sequences=True))

# Add a TimeDistributed Dense layer of size same as the vocabulary
model.add(TimeDistributed(Dense(len(vocabulary), activation='softmax')))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 13, 50)            5250      
_________________________________________________________________
time_distributed (TimeDistri (None, 13, 54)            2754      
Total params: 8,004
Trainable params: 8,004
Non-trainable params: 0
_________________________________________________________________


##### Built and compiled the recurrent neural network model successfully! This model can be trained now.

### Train RNN model and start predictions
- The output name will be generated character by character. The first character in each name was the start token \t. We'll feed the start token to the trained model to output a probability distribution over the vocabulary which can be sampled to generate the next character.

In [12]:
# Fit the model for 5 epochs using a batch size of 128 
model.fit(input_data, target_data, batch_size=128, epochs=5)

# Create a 3-D zero vector and initialize it with the start token
output_seq = np.zeros((1, max_len+1, len(vocabulary)))
output_seq[0, 0, char_to_idx['\t']] = 1

Epoch 1/5
2016/2016 [==============================] - 17s 9ms/step - loss: 1.1450
Epoch 2/5
2016/2016 [==============================] - 15s 7ms/step - loss: 1.0045
Epoch 3/5
2016/2016 [==============================] - 15s 7ms/step - loss: 0.9647
Epoch 4/5
2016/2016 [==============================] - 14s 7ms/step - loss: 0.9393
Epoch 5/5
2016/2016 [==============================] - 14s 7ms/step - loss: 0.9218


In [13]:
probs = model.predict_proba(output_seq, verbose=0)

W0106 23:04:19.637348 11992 deprecation.py:323] From <ipython-input-13-63d352476d46>:1: Sequential.predict_proba (from tensorflow.python.keras.engine.sequential) is deprecated and will be removed after 2021-01-01.
Instructions for updating:
Please use `model.predict()` instead.


In [14]:
probs

array([[[5.05111893e-06, 1.40933131e-04, 8.54209065e-02, 4.80010808e-02,
         8.72696936e-02, 6.19037561e-02, 6.96391016e-02, 2.32849047e-02,
         3.89882736e-02, 3.22241895e-02, 1.82152372e-02, 7.73006305e-02,
         3.65120210e-02, 8.26131925e-02, 8.02469328e-02, 2.15875525e-02,
         1.79130249e-02, 2.16133650e-02, 2.31377152e-03, 5.74634708e-02,
         5.02044857e-02, 3.63153443e-02, 2.14752415e-03, 1.88038312e-02,
         2.04673074e-02, 3.96588439e-04, 2.52892659e-03, 4.95911064e-03,
         1.90908868e-05, 5.47983545e-05, 5.10593200e-05, 7.32040326e-06,
         8.39324275e-06, 2.51117817e-05, 1.14214854e-04, 8.11769351e-05,
         8.36586114e-05, 9.77254385e-05, 2.88685515e-05, 7.13293703e-05,
         2.83754889e-05, 3.29365685e-05, 5.61140187e-05, 1.56319758e-04,
         5.09298879e-05, 4.78555849e-05, 8.55504004e-06, 2.60332981e-05,
         5.41282825e-05, 2.60669876e-05, 1.38620264e-04, 3.24685607e-05,
         1.91080180e-04, 2.75551829e-05],
        [

In [15]:
# Get the probabilities for the first character
probs = model.predict_proba(output_seq, verbose=0)[:,1,:] # 1st row

In [16]:
probs

array([[7.41022995e-06, 5.29358862e-04, 1.17851901e-04, 9.90898407e-05,
        1.20020661e-04, 1.62850411e-04, 8.72691016e-05, 7.70141269e-05,
        8.33229642e-05, 6.64390973e-05, 8.83352841e-05, 1.16792777e-04,
        1.07943531e-04, 9.81736739e-05, 1.09242865e-04, 6.64849867e-05,
        7.23943886e-05, 7.47506492e-05, 3.11992480e-05, 1.10546185e-04,
        1.04984516e-04, 7.90059057e-05, 3.49936563e-05, 8.11532154e-05,
        6.70688314e-05, 3.16388650e-05, 4.76981877e-05, 5.51156772e-05,
        2.87795633e-01, 1.85439526e-03, 7.87603203e-04, 6.19071827e-04,
        1.47231668e-01, 2.92595214e-04, 1.54598310e-04, 1.48100555e-02,
        1.27019167e-01, 1.26569517e-04, 2.87087809e-04, 1.29838716e-02,
        3.16271302e-03, 1.42107997e-03, 1.79781556e-01, 9.14728967e-04,
        9.81631456e-05, 3.64420824e-02, 1.36314763e-03, 5.97167236e-04,
        1.52111232e-01, 1.34334236e-03, 2.87267775e-03, 2.08864309e-04,
        2.24103983e-02, 5.82315552e-04]], dtype=float32)

### numpy.random.choice

```
 numpy.random.choice(a, size=None, replace=True, p=None)¶
 
Parameters

    a :1-D array-like or int
        If an ndarray, a random sample is generated from its elements. If an int, the random sample is generated as if a were np.arange(a) sizeint or tuple of ints, optional

Output shape : If the given shape is, e.g., (m, n, k), then m * n * k samples are drawn. Default is None, in which case a single value is returned.
    replace: boolean, optional

        Whether the sample is with or without replacement
    p : 1-D array-like, optional

        The probabilities associated with each entry in a. If not given the sample assumes a uniform distribution over all entries in a.
```


In [17]:
# Sample vocabulary to get first character
first_char = np.random.choice(sorted(vocabulary), replace=False, p=probs.reshape(len(vocabulary)))

# Print the character genaerated
print(first_char)

a


##### Now we know how to train the RNN model and generate the first character given the seed character as input. We'll use this character to generate the next character

### Generate baby names
- generate the second character by feeding the start token and the generated first character again to the trained network. We'll also be generating full names starting from the start token and repeating this process until the end token is found.

In [18]:
# Print the first character which we got last time
print(first_char)

# Update the vector to contain first the character
output_seq[0, 1, char_to_idx[first_char]] = 1

# Get the probabilities for the second character
probs = model.predict_proba(output_seq, verbose=0)[:,2,:]

# Sample vocabulary to get second character
second_char = np.random.choice(sorted(vocabulary), replace=False, p=probs.reshape(len(vocabulary)))

# Print the second character
print(second_char)

a
d


In [19]:
def generate_baby_names(n):
    for i in range(0,n):
        stop=False
        counter=1
        name=''
        # initialize first char of output seq
        output_seq=np.zeros((1,max_len+1,len(vocabulary)))
        output_seq[0,0,char_to_idx['\t']] = 1
        # continue until a newline is generated or max no of chars reached
        while stop==False and counter<10:
            # get prob distribution for next char
            probs = model.predict_proba(output_seq, verbose=0)[:,counter-1,:]
            # sample vocab to get most probable next char
            c = np.random.choice(sorted(list(vocabulary)), replace=False, p=probs.reshape(len(vocabulary)))
            if c=='\n':
                stop=True
            else:
                name = name + c
                output_seq[0,counter,char_to_idx[c]] = 1
                counter+= 1
        print(name)

In [20]:
generate_baby_names(5)

Daciey
Corra
Zannart
Ulbith
Jeathin


### Simple network using Keras
- We know how the gradient values become lesser and lesser as we back-propagate.
- demonstrate this vanishing gradient problem. We'll create a **simple network of Dense layers using Keras and checkout the gradient values of the weights for one iteration of back-propagation**

In [2]:
from keras import backend
import tensorflow as tf
import numpy as np

# The TensorFlow 2.0 has enabled eager execution by default. At the starting of algorithm, you need to use tf.compat.v1.disable_eager_execution() to disable eager execution. 
tf.compat.v1.disable_eager_execution()

# Create a sequential model
model = Sequential()

# Create a dense layer of 12 units
model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='relu'))

# Create a dense layer of 8 units
model.add(Dense(8, kernel_initializer='uniform', activation='relu'))

# Create a dense layer of 1 unit
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

# Compile the model and get gradients
model.compile(loss='binary_crossentropy', optimizer="adam")
with tf.GradientTape() as gtape:
    gradients = gtape.gradient(model.output, model.trainable_weights)

W0106 23:11:22.467451 23368 deprecation.py:506] From C:\Users\Shubham\Anaconda3\lib\site-packages\tensorflow\python\ops\resource_variable_ops.py:1666: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


##### Now we have a toy network to test gradient values of weights from each layer. Checking gradient values from successive layers is crucial to identify vanishing or exploding gradient problems.

### Vanishing gradients
- Before we start working on more robust applications of language generation, it's best to learn how to identify when you are suffering from the vanishing gradient problem.
- We will train the network created using random training data. 
- To run a model in `Tensorflow`, we need to initialize a tensorflow session first by using the `InteractiveSession()` function. Then we will initialize all variables using the `global_variables_initializer()` function.
- We will execute gradients node which we created inside this tensorflow session. We will also check some of the gradient values from different layers.

In [3]:
# Create a dummy input vector
input_vector = np.random.random((1,8))

# Create a tensorflow session to run the network
sess = tf.compat.v1.InteractiveSession()

# Initialize all the variables
sess.run(tf.compat.v1.global_variables_initializer())

# Evaluate the gradients using the training examples
evaluated_gradients = sess.run(gradients,feed_dict={model.input:input_vector})

# Print gradient values from third layer and two nodes of the second layer
print(evaluated_gradients[4])
print(evaluated_gradients[2][4])

TypeError: Fetch argument None has invalid type <class 'NoneType'>